# 3rd Question

### Which player has the highest number of wins?

Installing needed packages

In [ ]:
%pip install numpy pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


Importing needed packages

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os

# Reading .parquet data
According to question and ERD in /data/ERD folder as Tennis Schema.pdf , We are analyzing which player has the highest win rate, Reading data of matchs
It might take few seconds to read data

In [22]:
days_to_query = os.listdir("../data/raw/tennis_data")
files_to_load = []

for day in days_to_query:
    files_to_load.extend(
        glob(
            f"../data/raw/tennis_data/{day}/data/raw/raw_statistics*/statistics_*.parquet"
        )
    )

tennis_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=[
                "match_id",
                "period",
                "statistic_name",
                "home_stat",
                "away_stat",
                "home_total",
                "away_total",
            ],
        )
        for file in files_to_load
    ],
    ignore_index=True,
)

,match_id,period,statistic_name,home_stat,away_stat,home_total,away_total
0,11998445,ALL,aces,12,6,NaN,NaN
1,11998445,ALL,double_faults,2,7,NaN,NaN
2,11998445,ALL,first_serve,57/101 (56%),53/90 (59%),101.0,90.0
3,11998445,ALL,second_serve,42/44 (95%),30/37 (81%),44.0,37.0
4,11998445,ALL,first_serve_points,42/57 (74%),39/53 (74%),57.0,53.0
...,...,...,...,...,...,...,...
95,11998446,1ST,first_serve_points,12/18 (67%),12/16 (75%),18.0,16.0
96,11998446,1ST,second_serve_points,7/11 (64%),9/12 (75%),11.0,12.0
97,11998446,1ST,service_games_played,5,5,NaN,NaN
98,11998446,1ST,break_points_saved,0/1 (0%),2/2 (100%),1.0,2.0


### Reading data of Matches and Teams

In [ ]:
days_to_query_ = os.listdir("../data/raw/tennis_data")
files_to_load_ = []

for day in days_to_query_:
    files_to_load_.extend(
        glob(f"../data/raw/tennis_data/{day}/data/raw/raw_match*/home_team_1*.parquet")
    )

home_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=["match_id", "name"],
        )
        for file in files_to_load_
    ],
    ignore_index=True,
)

In [ ]:
days_to_query__ = os.listdir("../data/raw/tennis_data")
files_to_load__ = []

for day in days_to_query__:
    files_to_load__.extend(
        glob(f"../data/raw/tennis_data/{day}/data/raw/raw_match*/away_team_1*.parquet")
    )

away_statistics_data_frame = pd.concat(
    [
        pd.read_parquet(
            file,
            columns=["match_id", "name"],
        )
        for file in files_to_load__
    ],
    ignore_index=True,
)

Display data of home matches required information :

In [44]:
home_statistics_data_frame.rename(
    columns={"name": "Home Player", "match_id": "Match_id"}, inplace=True
)
home_statistics_data_frame.head()

,Match_id,Home Player
0,11998445,Cazaux A.
1,11998446,Lestienne C.
2,11998447,Ćorić B.
3,11998448,Mmoh M.
4,11998449,Paire B.


Display data of away matches required information :

In [46]:
away_statistics_data_frame.rename(
    columns={"name": "Away Player", "match_id": "Match_id"}, inplace=True
)
away_statistics_data_frame.head()

,Match_id,Away Player
0,11998445,Auger-Aliassime F.
1,11998446,Cobolli F.
2,11998447,Martínez P.
3,11998448,Muller A.
4,11998449,Mayot H.


Merging whole data

In [83]:
merged_df = pd.merge(
    home_statistics_data_frame, away_statistics_data_frame, on="Match_id", how="inner"
)
merged_df.rename(
    columns={"Home_Player_Name": "Home_Player", "Away_Player_Name": "Away_Player"},
    inplace=True,
)
merged_df_unique = merged_df.drop_duplicates(subset=["Match_id"], keep="last")
merged_df_unique.head(10)

,Match_id,Home Player,Away Player
6,11998448,Mmoh M.,Muller A.
7,11998449,Paire B.,Mayot H.
8,11998450,Shapovalov D.,Bublik A.
9,11998451,Shevchenko A.,Barrere G.
23,11998672,Imai S.,Sekulic P.
24,11998674,Saville L.,Jones H.
25,11998675,Hijikata R.,Jasika O.
26,11998676,Noguchi R.,Sweeny D.
29,11998774,Duckworth J.,Maloney P.
32,11998782,Sandgren T.,Fanselow S.


In [88]:
filtered_df = tennis_statistics_data_frame[
    (tennis_statistics_data_frame["period"] == "ALL")
    & (tennis_statistics_data_frame["statistic_name"] == "total")
]
filtered_df.rename(columns={"match_id": "Match_id"}, inplace=True)
filtered_df.drop_duplicates(subset=["Match_id"], keep="first", inplace=True)
filtered_df.head(10)

C:\Users\Shahab\AppData\Local\Temp\ipykernel_19492\3177724849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={"match_id" : "Match_id"},inplace=True)
C:\Users\Shahab\AppData\Local\Temp\ipykernel_19492\3177724849.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop_duplicates(subset=['Match_id'], keep='first', inplace=True)


,Match_id,period,statistic_name,home_stat,away_stat,home_total,away_total
8,11998445,ALL,total,95,96,NaN,NaN
79,11998446,ALL,total,46,63,NaN,NaN
133,11998447,ALL,total,66,48,NaN,NaN
186,11998448,ALL,total,63,48,NaN,NaN
240,11998449,ALL,total,47,62,NaN,NaN
294,11998450,ALL,total,100,107,NaN,NaN
364,11998451,ALL,total,94,92,NaN,NaN
435,11998456,ALL,total,90,103,NaN,NaN
506,11998459,ALL,total,70,58,NaN,NaN
560,11998666,ALL,total,72,80,NaN,NaN


In [90]:
final_df = pd.merge(merged_df_unique, filtered_df, on="Match_id", how="inner")
final_df.rename(
    columns={"Home_Player_Name": "Home_Player", "Away_Player_Name": "Away_Player"},
    inplace=True,
)
final_df.head(25)

,Match_id,Home Player,Away Player,period,statistic_name,home_stat,away_stat,home_total,away_total
0,11998448,Mmoh M.,Muller A.,ALL,total,63,48,NaN,NaN
1,11998449,Paire B.,Mayot H.,ALL,total,47,62,NaN,NaN
2,11998450,Shapovalov D.,Bublik A.,ALL,total,100,107,NaN,NaN
3,11998451,Shevchenko A.,Barrere G.,ALL,total,94,92,NaN,NaN
4,11998672,Imai S.,Sekulic P.,ALL,total,126,112,NaN,NaN
5,11998674,Saville L.,Jones H.,ALL,total,111,95,NaN,NaN
6,11998675,Hijikata R.,Jasika O.,ALL,total,106,111,NaN,NaN
7,11998676,Noguchi R.,Sweeny D.,ALL,total,59,76,NaN,NaN
8,11998774,Duckworth J.,Maloney P.,ALL,total,65,45,NaN,NaN
9,11998782,Sandgren T.,Fanselow S.,ALL,total,73,54,NaN,NaN


In [107]:
final_df.rename(
    columns={"home_stat": "Home Score", "away_stat": "Away Score"}, inplace=True
)
final_df.head(25)

,Match_id,Home Player,Away Player,statistic_name,Home Score,Away Score
0,11998448,Mmoh M.,Muller A.,total,63,48
1,11998449,Paire B.,Mayot H.,total,47,62
2,11998450,Shapovalov D.,Bublik A.,total,100,107
3,11998451,Shevchenko A.,Barrere G.,total,94,92
4,11998672,Imai S.,Sekulic P.,total,126,112
5,11998674,Saville L.,Jones H.,total,111,95
6,11998675,Hijikata R.,Jasika O.,total,106,111
7,11998676,Noguchi R.,Sweeny D.,total,59,76
8,11998774,Duckworth J.,Maloney P.,total,65,45
9,11998782,Sandgren T.,Fanselow S.,total,73,54


In [120]:
final_df["Result"] = np.where(
    final_df["Home Score"] > final_df["Away Score"], "Home_Won", "Away_Won"
)
final_df.head(10)

AttributeError: module 'pandas' has no attribute 'where'

Determine winner of each match\
Count number of wins per player\
Find player with most wins

In [ ]:
final_df["Winner"] = final_df.apply(
    lambda row: (
        row["Home Player"]
        if row["Home Score"] > row["Away Score"]
        else row["Away Player"]
    ),
    axis=1,
)

win_counts = final_df["Winner"].value_counts()

most_wins = win_counts.idxmax()
most_wins_count = win_counts.max()

Popko D. 26


In [ ]:
print(
    "Final Result , the player with most win :",
    most_wins,
    "Game Won :",
    most_wins_count,
)

Final Result , the player with most win : Popko D. Game Won : 26
